## Problem klasyfikacji sekwencji

Dane to ciągi różnych długości, np. tekst, muzyka, film.
* Elementami ciągów są “obiekty bazowe”
  * Tekst - ciąg słów (ogólniej - tokenów)
  * Film - ciąg obrazów
  * Muzyka - ciąg dźwięków
* Uwaga 1: elementy ciągów są od siebie zależne!
* Uwaga 2: kolejność elementów jest istotna!
* Jak można pracować z ciągami?
  * Sprowadzić ciągi do reprezentacji wektorowej i użyć
  klasycznych metod uczenia maszynowego
  * Użyć metod dedykowanych do takich danych - na przykład sieci rekurencyjnych

## NLP Modele Sekwencyjne i ich zastosowania
Modelem sekwencyjnym nazwiemy model, który jako wejście otrzymuje sekwencję, ale nie musi zwracać sekwencji.

## Sieci rekurencyjne
Zasadą działania sieci rekurencyjnej jest przechowywanie wyjścia poprzedniego elementu i wykorzystania go w kolejnym kroku.

Dlaczego po prostu do sekwencji nie wykorzystać zwykłych gęstych sieci neuronowych?
* Nie są w stanie przetwarzać sekwencji o różnych długościach
* Biorą pod uwagę tylko aktualne dane wejściowe
* Nie zapamiętują informacji o poprzednich danych wejściowych

### RNN
Podstawową wersją sieci rekurencyjnej jest RNN(recurrent Neural Network)
$$ h_t = \text{tanh}(Wx_t+Uh_{t-1}+b) $$
![](Grafika/RNN-unrolled.png)

Zalety:
* Możliwość przetwarzania sekwencji o dowolnej długości
* Rozmiar modelu nie rośnie razem z długością sekwencji
* Bierze pod uwagę poprzednie stany

Wady:
* Wolne obliczenia
* Problem wykorzystywania bardzo odległych stanów
* Nie może brać pod uwagę przyszłych stanów

### Embeddingi słów
Przeanalizujmy co się dzieje w sieci, gdy
podajemy słowa w reprezentacji _one hot_:
$$ h_t = f(W^h \cdot h_{t-1}+W^x\cdot x_t +b)$$
Jeśli $x_t$ to _one-got_ z jedynką na pozycji i to 
$$W^x\cdot x_t=W^x \cdot [0,\dots, 1_i,\dots, 0]^T=W^x[:,i]$$
Zatem przekształcenie to jest równoważne wzięciu i'tej kolumny macierzy wag.

Czyli i-ta kolumna macierzy wag jest w pewnym sensie reprezentacją słowa i-tego.

Zatem pójdźmy krok dalej: stwórzmy dodatkową warstwę w sieci - macierz 
embeddingów EMB, zawierającą reprezentacje słów, które będą
przekazywane do wyliczenia stanu ukrytego
\begin{align*}
    &emb_t=EMB\cdot x_t=EMB[:,i]\\
    &h_t=f(W^h \cdot h_{t-1}+W^x\cdot emb_t +b)
\end{align*}

Embeddingi są parametrami sieci, ale
jednocześnie reprezentacją słów.
Oznacza to, że trenując sieć, uczymy

embeddingi, czyli uczymy się reprezentacji słów!

Modele sekwencyjne można podzielić na kilka przykładów
### One-to-many, wejście o długości jednostkowej, wyjście o długości > 1. 
Przykład: generacja tekstu
   
![](Grafika/rnn-one-to-many-ltr.png)

### Many-to-one
przykład klasyfikacja sentymentu

![](Grafika/rnn-many-to-one-ltr.png)

### Many-to-many (tyle samo wejść co wyjść)
przykład: NER(named entity recognition)
![](Grafika/rnn-many-to-many-same-ltr.png)

### Many-to-many
przykład tłumaczenie maszynowe

![](Grafika/rnn-many-to-many-different-ltr.png)

### Problem odległych informacji relewantnych
Rozważmy problem predykcji następnego słowa po “the clouds
are in the __”

Jest to dość proste zadanie, bo odpowiedź można łatwo
wywnioskować na podstawie tych kilku słów.

W takich przypadkach zwykła sieć RNN jest odpowiednią
strukturą.

![](Grafika/RNN-shorttermdepdencies.png)



Próba przewidzenia “[I grew up in France… . I speak fluent ___”
wymaga sięgnięcia wstecz dalej niż kilka słów.

Ostatnie słowa sugerują tylko, że następne słowo jest nazwą języka -
odgadnięcie, że chodzi o francuski, wymaga odnalezienia”France”.

W praktyce dystans do relewantnej informacji często jest duży, a w
miarę wzrostu tego dystansu, zwykła sieć RNN staje się niezdolna do
wyłapania tych zależności.

![](Grafika/RNN-longtermdependencies.png)

### Eksplodujący/Zanikający gradient
Problem ten często spotyka się podczas korzystania z RNN. Wynika to z tego, że podczas wyliczania gradientu przemnażamy przez siebie wielokrotnie gradienty dla danych chwil czasowych w związku z tym może on zacząć zanikać(jak przemnażamy małe wartości), albo "wybuchnąć"(jak przemnażamy duże wartości).

Jak sobie poradzić z tym problem?

## LSTM
Sieci Long Short Term Memory – zazwyczaj
krótko “LSTM”.
Hochreiter & Schmidhuber, 1997 (!)

Są szczególnym rodzajem sieci
rekurencyjnych zaprojektowane tak, aby
zwiększyć skuteczność wykrywania
długodystansowych zależności.

### RNN
![](Grafika/LSTM3-SimpleRNN.png)

### LSTM
![](Grafika/LSTM3-chain.png)
![](Grafika/LSTM2-notation.png)

Gdzie $\sigma$ to aktywacja sigmoid.

Zatem jak można zauważyć LSTM posiada dwie "ścieżki" pamięci. Pierwszą jest tak zwany "cell state"
![](Grafika/LSTM3-C-line.png)
Ze względu na to, że ma on tylko liniowe interakcje łatwo jest o przepływ informacji tą scieżką. LSTM ma możliwość usuwania i dodawania informacji do tej ścieżki, co jest decydowanie przez tak zwane bramki(gates). Decydują one o tym czy dana informacją powinna dalej przejść

![](Grafika/LSTM3-gate.png)

Ponieważ sigmoida zwraca wartości między 0 a 1 decyduje jak "dużo" informacji powinno przepłynąć dalej.

#### LSTM krok po kroku
W pierwszym kroku decydujemy jak wiele aktualnej informacji powinno zostać w cell state.

![](Grafika/LSTM3-focus-f.png)

Następnie decydujemy jak wiele nowej informacji powinniśmy dodać do cell state

![](Grafika/LSTM3-focus-i.png)

Dokonujemy aktualizacji cell state

![](Grafika/LSTM3-focus-C.png)

Na koniec wybieramy interesujące nas informacje z zakutalizowanego cell state, które zostaną zwrócone przez LSTM

![](Grafika/LSTM3-focus-o.png)

Istnieją jeszcze inne warianty LSTM np. wykorzystujące cell state do bramek

![](Grafika/LSTM3-var-peepholes.png)

i takie, które wykorzystują jedną bramkę do zapominania/dodawania informacji do cell state

![](Grafika/LSTM3-var-tied.png)


### GRU a LSTM
GRU jest kolejną siecią rekurencyjną, której celem jest rozwiązanie odległych relacji między momentami czasu

![](Grafika/gru.png)

### Stacked RNN

![](Grafika/stacked_RNN.png)

### Bidirectional RNN
Czasem może nas interesować nie tylko infromacja z lewej do prawej ale także w drugą stronę, np. podczas klasyfikacji tekstu, w związku z tym aby otrzymać Biderctional RNN łączymy wyniki z dwóch sieci RNN, jedna "czyta" od lewej do prawej, a druga w drugą stronę.

In [1]:
import tensorflow as tf


In [2]:
example_input = tf.ones((1, 12, 2))


In [3]:
example_input

<tf.Tensor: shape=(1, 12, 2), dtype=float32, numpy=
array([[[1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.]]], dtype=float32)>

In [4]:
example_inputs = tf.ones((1, 12, 2))
example_prices = tf.ones((1, 12, 1))


In [5]:
rnn_layer = tf.keras.layers.SimpleRNN(
    units=512,
    activation="tanh",
    return_sequences=False,
    return_state=False,
)
rnn_layer(example_input).shape


TensorShape([1, 512])

In [6]:
rnn_layer = tf.keras.layers.SimpleRNN(
    units=512,
    activation="tanh",
    dropout=0.0,
    recurrent_dropout=0.0,
    return_sequences=True,
    return_state=False,
)
rnn_layer(example_input).shape


TensorShape([1, 12, 512])

In [7]:
rnn_layer = tf.keras.layers.SimpleRNN(
    units=512,
    activation="tanh",
    dropout=0.0,
    recurrent_dropout=0.0,
    return_sequences=True,
    return_state=True,
)
output, hidden_state = rnn_layer(example_input)
print(type(output))
print(output.shape, hidden_state.shape)


<class 'tensorflow.python.framework.ops.EagerTensor'>
(1, 12, 512) (1, 512)


In [8]:
bidirectional_rnn = tf.keras.layers.Bidirectional(tf.keras.layers.SimpleRNN(
    units=512,
    activation="tanh",
    recurrent_dropout=0.0,
    return_sequences=True,
    return_state=False,
))
output = bidirectional_rnn(example_input)
print(type(output))
print(output.shape)


<class 'tensorflow.python.framework.ops.EagerTensor'>
(1, 12, 1024)


In [11]:
vocab_size=10
output_dim=30

In [12]:
embedding_layer = tf.keras.layers.Embedding(
    vocab_size,
    output_dim,
    embeddings_initializer="uniform",
    embeddings_regularizer=None,
    activity_regularizer=None,
    embeddings_constraint=None,
    mask_zero=False,
    input_length=None,
    **kwargs
)


NameError: name 'kwargs' is not defined

Do warstwy Embedding możemy podać przetrenowane wcześniej embeddingi. Muszą one być wcześniej przygotowane jako macierz o wymiarach (vocab_size x embedding_dim). Oraz i'ty wiersz, musi odpowiadać embeddingowi tokenu, który przekształacmy na liczbę `i`.

In [13]:
import numpy as np
import tensorflow as tf

In [14]:
pretrained_embedding_matrix = np.array(range(10*30)).reshape((10,30))

In [16]:
pretrained_embedding = tf.keras.layers.Embedding(
    10,
    30,
    embeddings_initializer=tf.keras.initializers.Constant(pretrained_embedding_matrix),
    embeddings_regularizer=None,
    activity_regularizer=None,
    embeddings_constraint=None,
    mask_zero=False,
    input_length=None,
)



In [17]:
pretrained_embedding(np.array([[0,9]]))

<tf.Tensor: shape=(1, 2, 30), dtype=float32, numpy=
array([[[  0.,   1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,
          10.,  11.,  12.,  13.,  14.,  15.,  16.,  17.,  18.,  19.,
          20.,  21.,  22.,  23.,  24.,  25.,  26.,  27.,  28.,  29.],
        [270., 271., 272., 273., 274., 275., 276., 277., 278., 279.,
         280., 281., 282., 283., 284., 285., 286., 287., 288., 289.,
         290., 291., 292., 293., 294., 295., 296., 297., 298., 299.]]],
      dtype=float32)>

In [18]:
lstm = tf.keras.layers.LSTM(
    units=512,
    activation="tanh",
    recurrent_dropout=0.0,
    return_sequences=False,
    return_state=False,
)

In [19]:
lstm(example_input).shape

TensorShape([1, 512])

In [20]:
lstm = tf.keras.layers.LSTM(
    units=512,
    activation="tanh",
    recurrent_dropout=0.0,
    return_sequences=True,
    return_state=False,
)
lstm(example_input).shape

TensorShape([1, 12, 512])

In [21]:
lstm = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(
    units=512,
    activation="tanh",
    recurrent_dropout=0.0,
    return_sequences=True,
    return_state=False,
))
lstm(example_input).shape

TensorShape([1, 12, 1024])

In [22]:
lstm = tf.keras.layers.LSTM(
    units=512,
    activation="tanh",
    recurrent_dropout=0.0,
    return_sequences=True,
    return_state=True,
)
out, cell_state, hidden_state = lstm(example_input)
out.shape, cell_state.shape, hidden_state.shape

(TensorShape([1, 12, 512]), TensorShape([1, 512]), TensorShape([1, 512]))

In [23]:
lstm(example_input, initial_state=[cell_state, hidden_state])

[<tf.Tensor: shape=(1, 12, 512), dtype=float32, numpy=
 array([[[-0.03040359,  0.05052163, -0.0114219 , ...,  0.01880071,
           0.04016161,  0.05248069],
         [-0.03107806,  0.05108547, -0.01205938, ...,  0.01996111,
           0.04064401,  0.05330994],
         [-0.03160809,  0.05153299, -0.0126333 , ...,  0.02097402,
           0.04100688,  0.05395285],
         ...,
         [-0.03309631,  0.05296824, -0.01508493, ...,  0.02506671,
           0.0418531 ,  0.05558472],
         [-0.03314754,  0.05304562, -0.01526771, ...,  0.02536547,
           0.04186686,  0.05561829],
         [-0.03318093,  0.05310643, -0.01542296, ...,  0.02562031,
           0.04187149,  0.05563284]]], dtype=float32)>,
 <tf.Tensor: shape=(1, 512), dtype=float32, numpy=
 array([[-3.31809260e-02,  5.31064272e-02, -1.54229552e-02,
          2.68803257e-02, -3.66238281e-02,  8.42820015e-03,
          1.88487489e-02,  4.96336594e-02,  1.32185938e-02,
          2.06017531e-02, -1.60628129e-02, -8.81140456e-0

In [24]:
rnn_layer = tf.keras.layers.GRU(
    units=512,
    activation="tanh",
    return_sequences=False,
    return_state=False,
)
rnn_layer(example_input).shape


TensorShape([1, 512])

In [25]:
rnn_layer = tf.keras.layers.GRU(
    units=512,
    activation="tanh",
    dropout=0.0,
    recurrent_dropout=0.0,
    return_sequences=True,
    return_state=False,
)
rnn_layer(example_input).shape


TensorShape([1, 12, 512])

In [26]:
rnn_layer = tf.keras.layers.GRU(
    units=512,
    activation="tanh",
    dropout=0.0,
    recurrent_dropout=0.0,
    return_sequences=True,
    return_state=True,
)
output, hidden_state = rnn_layer(example_input)
print(type(output))
print(output.shape, hidden_state.shape)


<class 'tensorflow.python.framework.ops.EagerTensor'>
(1, 12, 512) (1, 512)


In [27]:
bidirectional_rnn = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(
    units=512,
    activation="tanh",
    recurrent_dropout=0.0,
    return_sequences=True,
    return_state=False,
))
output = bidirectional_rnn(example_input)
print(type(output))
print(output.shape)


<class 'tensorflow.python.framework.ops.EagerTensor'>
(1, 12, 1024)


In [28]:
rnn_layer1 = tf.keras.layers.GRU(
    units=512,
    activation="tanh",
    dropout=0.0,
    recurrent_dropout=0.0,
    return_sequences=True,
    return_state=False,
)
rnn_layer2 = tf.keras.layers.GRU(
    units=512,
    activation="tanh",
    dropout=0.0,
    recurrent_dropout=0.0,
    return_sequences=False,
    return_state=False,
)
mod = tf.keras.Sequential()
mod.add(rnn_layer1)
mod.add(rnn_layer2)

In [29]:
mod(example_input).shape

TensorShape([1, 512])

In [30]:
rnn_layer1 = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(
    units=512,
    activation="tanh",
    dropout=0.0,
    recurrent_dropout=0.0,
    return_sequences=True,
    return_state=False,
))
rnn_layer2 = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(
    units=512,
    activation="tanh",
    dropout=0.0,
    recurrent_dropout=0.0,
    return_sequences=False,
    return_state=False,
))
mod = tf.keras.Sequential()
mod.add(rnn_layer1)
mod.add(rnn_layer2)

In [31]:
mod(example_input).shape

TensorShape([1, 1024])

### Case study: IMBD
Klasyfikacja tekstu przy użyciu RNN

In [18]:
from keras import utils

from keras.datasets import imdb


In [45]:
max_features = 1000
maxlen = 50
batch_size = 32
embedding_dims = 50
epochs = 3


In [46]:
import numpy as np


(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)


In [47]:
# (x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

print(len(x_train), "train sequences")
print(len(x_test), "test sequences")
print(x_train[:3])


25000 train sequences
25000 test sequences
[list([1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 2, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 2, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 2, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 2, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 2, 19, 178, 32])
 list([1, 194, 1153, 194,

Zwróćmy uwagę w powyższym, że ciągi zaczynają się zawsze od "1" - jest to oznaczenie początku zdania. Czyli "początek zdania" będzie miał swój embedding.

Standaryzacja długości sekwencji (Padding)

In [48]:
?sequence.pad_sequences

Object `sequence.pad_sequences` not found.


In [49]:
x_train = utils.pad_sequences(x_train, maxlen=maxlen, padding="pre")
x_test = utils.pad_sequences(x_test, maxlen=maxlen, padding="pre")

print("x_train shape:", x_train.shape)
print("x_test shape:", x_test.shape)
print(x_train[0])


x_train shape: (25000, 400)
x_test shape: (25000, 400)
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    1   14   22   16  

In [50]:
n_train = 3000
n_test = 500
x_train = x_train[:n_train]
y_train = y_train[:n_train]
x_test = x_test[:n_test]
y_test = y_test[:n_test]


In [54]:
y_train.shape


(3000,)

In [87]:
x_train.shape

(3000, 400)

### Zadania

### Zwykła sieć rekurencyjna ( z embeddingami)

In [19]:
from keras.models import Sequential
from keras.layers import (
    Dense,
    Dropout,
    Embedding,
    SimpleRNN,
    LSTM,
    Bidirectional,
    GRU
)

from keras.callbacks import EarlyStopping


In [122]:
vocab_size = max_features #jak wczytywalismy dane to ustawilismy ze maks 5000 slow
output_dim = 512

In [123]:
embedding_layer = tf.keras.layers.Embedding(
    max_features,
    50,
    embeddings_initializer="uniform",
    embeddings_regularizer=None,
    activity_regularizer=None,
    embeddings_constraint=None,
    mask_zero=False,
    input_length=None,
)

In [124]:
rnn_layer = tf.keras.layers.SimpleRNN(
    units=64,
    activation="tanh",
    return_sequences=False,
    return_state=False,
)

In [125]:
model = tf.keras.Sequential()
model.add(embedding_layer)
model.add(rnn_layer)
model.add(Dense(1, activation="sigmoid"))

In [126]:
model.summary()

Model: "sequential_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_13 (Embedding)    (None, None, 50)          250000    
                                                                 
 simple_rnn_17 (SimpleRNN)   (None, 64)                7360      
                                                                 
 dense_3 (Dense)             (None, 1)                 65        
                                                                 
Total params: 257,425
Trainable params: 257,425
Non-trainable params: 0
_________________________________________________________________


In [127]:
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
early_stopping = EarlyStopping(patience=2)

model.fit(
    x_train,
    y_train,
    epochs=epochs,
    callbacks=[early_stopping],
    batch_size=batch_size,)

Epoch 1/3
94/94 [==============================] - 4s 38ms/step - loss: 0.6962 - accuracy: 0.5140
Epoch 2/3
94/94 [==============================] - 4s 41ms/step - loss: 0.6344 - accuracy: 0.6997
Epoch 3/3
94/94 [==============================] - 4s 39ms/step - loss: 0.4467 - accuracy: 0.8753


In [128]:
model.evaluate(x_test, y_test, verbose=0)

[0.6938501000404358, 0.5839999914169312]

Od trenera

In [129]:
model = Sequential()

model.add(Embedding(max_features, 300))
model.add(SimpleRNN(256))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


model.summary()

early = EarlyStopping(patience=2)


Model: "sequential_19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_14 (Embedding)    (None, None, 300)         1500000   
                                                                 
 simple_rnn_18 (SimpleRNN)   (None, 256)               142592    
                                                                 
 dense_4 (Dense)             (None, 1)                 257       
                                                                 
Total params: 1,642,849
Trainable params: 1,642,849
Non-trainable params: 0
_________________________________________________________________


In [130]:
model.fit(
    x_train,
    y_train,
    epochs=epochs,
    callbacks=[early_stopping],
    batch_size=batch_size,)

Epoch 1/3
94/94 [==============================] - 30s 314ms/step - loss: 0.7101 - accuracy: 0.5087
Epoch 2/3
94/94 [==============================] - 29s 307ms/step - loss: 0.6815 - accuracy: 0.5697
Epoch 3/3
94/94 [==============================] - 29s 311ms/step - loss: 0.6416 - accuracy: 0.6410


In [131]:
model.evaluate(x_test, y_test, verbose=0)

[0.6722758412361145, 0.5339999794960022]

### Simple RNN + dense pomiędzy zwracanym wyjściem z RNN a outputem

In [132]:
model = Sequential()

model.add(Embedding(max_features, 300))
model.add(SimpleRNN(256))
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

early = EarlyStopping(patience=2)

model.fit(
    x_train,
    y_train,
    epochs=epochs,
    callbacks=[early_stopping],
    batch_size=batch_size, )
model.evaluate(x_test, y_test, verbose=0)

Model: "sequential_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_15 (Embedding)    (None, None, 300)         1500000   
                                                                 
 simple_rnn_19 (SimpleRNN)   (None, 256)               142592    
                                                                 
 dense_5 (Dense)             (None, 256)               65792     
                                                                 
 dense_6 (Dense)             (None, 1)                 257       
                                                                 
Total params: 1,708,641
Trainable params: 1,708,641
Non-trainable params: 0
_________________________________________________________________
Epoch 1/3
94/94 [==============================] - 27s 276ms/step - loss: 0.7448 - accuracy: 0.5090
Epoch 2/3
94/94 [==============================] - 38s 403ms/step - loss: 0

[0.7444236278533936, 0.49000000953674316]

## Dwukierunkowa sieć rekurencyjna

In [133]:
bidirectional_rnn = tf.keras.layers.Bidirectional(tf.keras.layers.SimpleRNN(units=512))
#                                                   , activation="tanh",
#     recurrent_dropout=0.0,
#     return_sequences=True,
#     return_state=False,
# ))
# output = bidirectional_rnn(example_input)
# print(type(output))
# print(output.shape)

In [134]:
model = Sequential()

model.add(Embedding(max_features, 300))
model.add(bidirectional_rnn)
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

early = EarlyStopping(patience=2)

Model: "sequential_21"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_16 (Embedding)    (None, None, 300)         1500000   
                                                                 
 bidirectional_5 (Bidirectio  (None, 1024)             832512    
 nal)                                                            
                                                                 
 dense_7 (Dense)             (None, 1)                 1025      
                                                                 
Total params: 2,333,537
Trainable params: 2,333,537
Non-trainable params: 0
_________________________________________________________________


In [135]:
model.fit(
    x_train,
    y_train,
    epochs=epochs,
    callbacks=[early_stopping],
    batch_size=batch_size, )

Epoch 1/3
94/94 [==============================] - 82s 856ms/step - loss: 0.7252 - accuracy: 0.5190
Epoch 2/3
94/94 [==============================] - 84s 897ms/step - loss: 0.7161 - accuracy: 0.5023
Epoch 3/3
94/94 [==============================] - 91s 963ms/step - loss: 0.6947 - accuracy: 0.5357


In [136]:
model.evaluate(x_test, y_test, verbose=0)

[0.6811016201972961, 0.5799999833106995]

### Zadanie. Powtórz powyższe modele z komórką LSTM

In [137]:
lstm = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units=512))

In [138]:
model = Sequential()

model.add(Embedding(max_features, 300))
model.add(lstm)
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

early = EarlyStopping(patience=2)

Model: "sequential_22"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_17 (Embedding)    (None, None, 300)         1500000   
                                                                 
 bidirectional_6 (Bidirectio  (None, 1024)             3330048   
 nal)                                                            
                                                                 
 dense_8 (Dense)             (None, 1)                 1025      
                                                                 
Total params: 4,831,073
Trainable params: 4,831,073
Non-trainable params: 0
_________________________________________________________________


In [139]:
model.fit(
    x_train,
    y_train,
    epochs=epochs,
    callbacks=[early_stopping],
    batch_size=batch_size, )

Epoch 1/3
94/94 [==============================] - 251s 3s/step - loss: 0.6975 - accuracy: 0.5737
Epoch 2/3
94/94 [==============================] - 266s 3s/step - loss: 0.5326 - accuracy: 0.7510
Epoch 3/3
94/94 [==============================] - 284s 3s/step - loss: 0.3044 - accuracy: 0.8703


In [140]:
model.evaluate(x_test, y_test, verbose=0)

[0.5122829675674438, 0.7839999794960022]

## Dwuwarstwowa sieć rekurencyjna

In [141]:
rnn_layer1 = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(
    units=512,
    activation="tanh",
    dropout=0.0,
    recurrent_dropout=0.0,
    return_sequences=True,
    return_state=False,
))
rnn_layer2 = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(
    units=512,
    activation="tanh",
    dropout=0.0,
    recurrent_dropout=0.0,
    return_sequences=False,
    return_state=False,
))

In [142]:
model = Sequential()

model.add(Embedding(max_features, 300))
model.add(rnn_layer1)
model.add(rnn_layer2)
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

early = EarlyStopping(patience=2)

Model: "sequential_23"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_18 (Embedding)    (None, None, 300)         1500000   
                                                                 
 bidirectional_7 (Bidirectio  (None, None, 1024)       2500608   
 nal)                                                            
                                                                 
 bidirectional_8 (Bidirectio  (None, 1024)             4724736   
 nal)                                                            
                                                                 
 dense_9 (Dense)             (None, 1)                 1025      
                                                                 
Total params: 8,726,369
Trainable params: 8,726,369
Non-trainable params: 0
_________________________________________________________________


In [143]:
model.fit(
    x_train,
    y_train,
    epochs=epochs,
    callbacks=[early_stopping],
    batch_size=batch_size, )

Epoch 1/3
94/94 [==============================] - 560s 6s/step - loss: 0.7120 - accuracy: 0.5893
Epoch 2/3
94/94 [==============================] - 622s 7s/step - loss: 0.4520 - accuracy: 0.7937
Epoch 3/3
67/94 [====================>.........] - ETA: 3:04 - loss: 0.2427 - accuracy: 0.9025

KeyboardInterrupt: 

In [ ]:
model.evaluate(x_test, y_test, verbose=0)

### CNN a tekst

Mając sekwencję możemy zamiast wykorzystywać RNN skorzystać z jednowymiarowej konwolucji. W tym przypadku rozmiar kernela(jądra) decyduje o tym na ile chwil czasowych jednocześnie patrzy CNN. 

In [ ]:
from keras.layers import Conv1D, GlobalAveragePooling1D, GlobalAveragePooling2D

In [ ]:
GlobalAveragePooling2D()(tf.zeros((16, 32,32,3))).shape


#### Zadanie
korzystając z warstwy `tf.keras.layers.Conv1D` i jakiejś warstwy poolingowej zastąpić w poprzednim modelu RNN siecią CNN.
Argumenty są takie same jak dla konwolucji 2D tylko `kernel_size` i `strides` mogą być tylko liczbami całkowitymi.

In [3]:
from keras.layers import Conv1D, GlobalAveragePooling1D, GlobalAveragePooling2D

In [4]:
GlobalAveragePooling2D()(tf.zeros((16, 32,32,3))).shape


2022-07-08 15:25:08.127780: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:961] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-07-08 15:25:08.171478: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:961] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-07-08 15:25:08.172010: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:961] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-07-08 15:25:08.173406: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate

TensorShape([16, 3])

In [ ]:
from keras.layers import Conv1D, GlobalAveragePooling1D, GlobalAveragePooling2D

model = Sequential()
model.add(Embedding(max_features, 64, input_length=maxlen))

model.add(Conv1D(32, kernel_size=5))
model.add(Conv1D(64, kernel_size=5))
model.add(Conv1D(128, kernel_size=5))
model.add(GlobalAveragePooling1D())
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation="sigmoid"))

model.compile(
    loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"]
)
model.summary()

model.fit(
    x_train,
    y_train,
    batch_size=batch_size,
    epochs=100,
    callbacks=[early],
    validation_split=0.10,
)

### Case Study
Utwórz model, który identyfikuje emocje wpisu na twiterze.

In [20]:
import tensorflow as tf
import numpy as np
import pandas as pd
import re

In [21]:
data = pd.read_csv('Dane/tweet_emotions.csv')

In [22]:
data.head()

,tweet_id,sentiment,content
0,1956967341,empty,@tiffanylue i know i was listenin to bad habi...
1,1956967666,sadness,Layin n bed with a headache ughhhh...waitin o...
2,1956967696,sadness,Funeral ceremony...gloomy friday...
3,1956967789,enthusiasm,wants to hang out with friends SOON!
4,1956968416,neutral,@dannycastillo We want to trade with someone w...


In [23]:
X = data['content'].tolist()
y = data['sentiment']

In [24]:
def preprocess_tweets(sent):
    outsent = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",sent).split())
    outsent = outsent.lower()
    return outsent

import re

X = [preprocess_tweets(elem) for elem in X]


# Tworzymy warstwe tokenizującą
vectorize_layer = tf.keras.layers.TextVectorization(
    max_tokens=5000, output_mode="int"
)
# Dopasujemy tokenizator do danych
vectorize_layer.adapt(X) #tworzy słownik


from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_label = le.fit_transform(y)

from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(np.array(X), y_label, test_size=0.1)

In [27]:
max_features = 5000
maxlen = 50
batch_size = 32
embedding_dims = 50
epochs = 3
early = EarlyStopping(patience=2)

In [32]:
from keras.layers import Conv1D, GlobalAveragePooling1D, GlobalAveragePooling2D

model = Sequential()
model.add(vectorize_layer)
model.add(Embedding(max_features, 64, input_length=10))

model.add(Conv1D(256, 5))
# model.add(Conv1D(64, kernel_size=5))
# model.add(Conv1D(128, kernel_size=5))
model.add(GlobalAveragePooling1D())
model.add(Dense(64, activation='relu'))
model.add(Dense(13, activation="sigmoid"))

model.compile(
    loss=tf.keras.losses.sparse_categorical_crossentropy, optimizer="adam", metrics=["sparse_categorical_accuracy"]
)
model.summary()

model.fit(
    x_train,
    y_train,
    batch_size=batch_size,
    epochs=100,
    callbacks=[early],
    validation_split=0.10,
)

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization_2 (TextV  (None, None)             0         
 ectorization)                                                   
                                                                 
 embedding_3 (Embedding)     (None, None, 64)          320000    
                                                                 
 conv1d_7 (Conv1D)           (None, None, 256)         82176     
                                                                 
 global_average_pooling1d_3   (None, 256)              0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_6 (Dense)             (None, 64)                16448     
                                                                 
 dense_7 (Dense)             (None, 13)               

In [33]:
model.evaluate(x_test, y_test, verbose=0)

[2.0025696754455566, 0.3330000042915344]

In [34]:
bidirectional_rnn = tf.keras.layers.Bidirectional(tf.keras.layers.SimpleRNN(units=512))
from keras.layers import Conv1D, GlobalAveragePooling1D, GlobalAveragePooling2D

model = Sequential()
model.add(vectorize_layer)
model.add(Embedding(max_features, 64, input_length=10))

model.add(bidirectional_rnn)
model.add(Dense(64, activation='relu'))
model.add(Dense(13, activation="sigmoid"))

model.compile(
    loss=tf.keras.losses.sparse_categorical_crossentropy, optimizer="adam", metrics=["sparse_categorical_accuracy"]
)
model.summary()

model.fit(
    x_train,
    y_train,
    batch_size=batch_size,
    epochs=100,
    callbacks=[early],
    validation_split=0.10,
)

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization_2 (TextV  (None, None)             0         
 ectorization)                                                   
                                                                 
 embedding_4 (Embedding)     (None, None, 64)          320000    
                                                                 
 bidirectional (Bidirectiona  (None, 1024)             590848    
 l)                                                              
                                                                 
 dense_8 (Dense)             (None, 64)                65600     
                                                                 
 dense_9 (Dense)             (None, 13)                845       
                                                                 
Total params: 977,293
Trainable params: 977,293
Non-tr

In [35]:
model.evaluate(x_test, y_test, verbose=0)

[2.1383774280548096, 0.2175000011920929]

In [36]:
rnn_layer1 = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(
    units=512,
    activation="tanh",
    dropout=0.0,
    recurrent_dropout=0.0,
    return_sequences=True,
    return_state=False,
))
rnn_layer2 = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(
    units=512,
    activation="tanh",
    dropout=0.0,
    recurrent_dropout=0.0,
    return_sequences=False,
    return_state=False,
))

In [39]:
model = Sequential()
model.add(vectorize_layer)
model.add(Embedding(max_features, 64, input_length=10))

model.add(rnn_layer1)
model.add(rnn_layer2)
model.add(Dense(64, activation='relu'))
model.add(Dense(13, activation="sigmoid"))

model.compile(
    loss=tf.keras.losses.sparse_categorical_crossentropy, optimizer="adam", metrics=["sparse_categorical_accuracy"]
)
model.summary()

model.fit(
    x_train,
    y_train,
    batch_size=batch_size,
    epochs=5,
    callbacks=[early],
    validation_split=0.10,
)

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization_2 (TextV  (None, None)             0         
 ectorization)                                                   
                                                                 
 embedding_7 (Embedding)     (None, None, 64)          320000    
                                                                 
 bidirectional_1 (Bidirectio  (None, None, 1024)       1775616   
 nal)                                                            
                                                                 
 bidirectional_2 (Bidirectio  (None, 1024)             4724736   
 nal)                                                            
                                                                 
 dense_14 (Dense)            (None, 64)                65600     
                                                      

In [40]:
model.evaluate(x_test, y_test, verbose=0)

[1.9611717462539673, 0.3317500054836273]

#### Contextual embeddings
Przy użyciu sieci rekurencyjnych można tworzyć embeddingi, które uwzględniają kontekst w jakim użyte jest dane słowo. Istnieje wiele słów, które jak są wykorzystane bez kontekstu nie można jednoznacznie określić ich znaczenia(np. zamek jako budowla i jako zapięcie kurtki). W związku z tym wykorzystuje się modele sekwencyjne, które modelują słowo w zależności od jego "otoczenia"
##### ELMo
Model ELMo wykorzsytuje sieci Bidirectional RNN do reprezentacji kontekstu

![](Grafika/Bert-language-modeling.png)

Działanie ELMo

![](Grafika/elmo-forward-backward-language-model-embedding.png)

![](Grafika/elmo-embedding.png)

In [ ]:
import tensorflow_hub as hub


In [ ]:
elmo = hub.load("https://tfhub.dev/google/elmo/3").signatures["default"]


In [ ]:
x = ["ELMo lives on sesame street."]

# Extract ELMo features
embeddings = elmo(tf.constant(x))["elmo"]

embeddings.shape


In [ ]:
X_elmo_embeddings = elmo(tf.constant(X[:100]))["elmo"]

In [ ]:
X_elmo_embeddings.shape

In [ ]:
model_elmo = Sequential()
model_elmo.add(Bidirectional(GRU(512)))
model_elmo.add(Dense(512, activation="relu"))
model_elmo.add(Dropout(0.4))
model_elmo.add(Dense(num_classes, activation="softmax"))

In [ ]:
model_elmo.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    optimizer="adam",
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)

In [ ]:
model_elmo.fit(
    X_elmo_embeddings,
    y_label[:100],
    batch_size=batch_size,
    epochs=100,
    callbacks=[early_stopping],
    validation_split=0.10,
)

### Seq2Seq
Modele Seq2Seq składają się z dwóch modeli sekwencyjnych, zazwyczaj pierwszy nazywa się `encoder` a drugi `decoder`. Zazwyczaj sekwencje wejściowe i wyjściowe mogą być różnej długości.
Przykłady zastosowań
* Machine translation
* Table summarization
* Image captioning
* Document Summarization
* Question Answering(np. chatboty)
* Speech recognition

![](Grafika/seq2seq-teacher-forcing.png)

Aby utworzy takie modele trzeba już korzystać z functional API w Kerasie, ponieważ tutaj nie mamy prostego liniowego potoku zmiennych.

### Spacy i problemy lingwistyczne
spacy jest biblioteką zawierającą wiele modeli do problemów lingwistycznych, które teraz sobie krótko omówimy.

##### tokenizacja
spacy także posiada tokenizacje

In [ ]:
import spacy


In [ ]:
# wczytywanie modelu językowego
nlp = spacy.load("en_core_web_sm")
doc = nlp("Akash has been buyed by byju's in 73,000 Core's")
for token in doc:
    print(token.text)
    print(token.lemma_)


#### Part-Of_Speech (POS) - tagging
problem ten polega na wyjaśnieniu w jaki sposób dane słowo jest wykorzystane w zdaniu. Ustalone jest 8 części mowy (po ang bo będzie łatwiej):
* Noun
* Pronoun
* Adjective
* Verb
* Adverb
* Preposition
* Conjunction
* Interjection

In [ ]:
doc = nlp("We are currently learning about Spacy")

# Iterate over the tokens
for token in doc:
    # Print the token and its part-of-speech tag
    print(token, token.tag_, token.pos_, spacy.explain(token.tag_))


In [ ]:
doc = nlp("We are currently learning about Spacy")
spacy.displacy.render(doc, style="dep", jupyter=True)


#### Dependency Parsing
Jest to proces tworzenia struktury gramatycznej zdania. Daje on nam zależność słów w zdaniu.

In [ ]:
doc = nlp("We are currently learning about Spacy")

for token in doc:
    print(token.text, "-->", token.dep_)


#### Named Entity Recogniton
Czyli po prostu wykrywanie nazw własnych

In [ ]:
nlp = spacy.load("en_core_web_sm")
doc = nlp(
    "Reliance is looking at buying U.K. based analytics startup for $7 billion"
)
# See the entity present
print(f"Enitites: {doc.ents}")
for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)


#### Entity Recogniton
nazywane też Entity Detection jest bardziej zaawansowany od NER, ponieważ rozpoznaje istotne elementy między innymi miejsca, ludzi, organizacje, języki itp.

In [ ]:
nlp = spacy.load("en_core_web_sm")
doc = nlp(
    """The Amazon rainforest,[a] alternatively, the Amazon Jungle, also known in English as Amazonia, is a moist broadleaf tropical rainforest in the Amazon biome that covers most of the Amazon basin of South America. This basin encompasses 7,000,000 km2 (2,700,000 sq mi), of which 5,500,000 km2 (2,100,000 sq mi) are covered by the rainforest. This region includes territory belonging to nine nations.

The majority of the forest is contained within Brazil, with 60% of the rainforest, followed by Peru with 13%, Colombia with 10%, and with minor amounts in Bolivia, Ecuador, French Guiana, Guyana, Suriname, and Venezuela. Four nations have "Amazonas" as the name of one of their first-level administrative regions and France uses the name "Guiana Amazonian Park" for its rainforest protected area. The Amazon represents over half of the planet's remaining rainforests,[2] and comprises the largest and most biodiverse tract of tropical rainforest in the world, with an estimated 390 billion individual trees divided into 16,000 species.[3]

Etymology
The name Amazon is said to arise from a war Francisco de Orellana fought with the Tapuyas and other tribes. The women of the tribe fought alongside the men, as was their custom.[4] Orellana derived the name Amazonas from the Amazons of Greek mythology, described by Herodotus and Diodorus.[4]

History
See also: History of South America § Amazon, and Amazon River § History
Tribal societies are well capable of escalation to all-out wars between tribes. Thus, in the Amazonas, there was perpetual animosity between the neighboring tribes of the Jivaro. Several tribes of the Jivaroan group, including the Shuar, practised headhunting for trophies and headshrinking.[5] The accounts of missionaries to the area in the borderlands between Brazil and Venezuela have recounted constant infighting in the Yanomami tribes. More than a third of the Yanomamo males, on average, died from warfare.[6]"""
)

entities = [(i, i.label_, i.label) for i in doc.ents]
entities


In [ ]:
spacy.displacy.render(doc, style="ent", jupyter=True)


#### Transformers
Transformery wywołały ogromny przełom w NLP, są one w stanie modelować relacje między dowolnie odległymi chwilami czasu i są szybsze ponieważ nie wymagają pętli(wystarcza tylko mnożenie wektorów i macierzy)

![](Grafika/transformer.png)

Czym jest `Positional Encoding`? Istnieją dwie szkoły tworzenia
* Uczymy embeddingi pozycji razem z modelem, czyli `Positional Encoding` jest parameterem modelu o ustalonej długości
* Wykorzystujemy z góry zdefiniowaną funkcję np.
\begin{align*}
  p_{i,j}=
  \begin{cases}
    \sin\left(\frac{i}{10000^{j/d}}\right)\\
    \cos\left(\frac{i}{10000^{(j-1)/d}}\right)
  \end{cases}
\end{align*}
gdzie $i$-chwila czasu, $j$-j'ty wymiar w wektorze positional encodingu, $d$-wymiar positional encodingu.

Positional Encoding jest potrzebny, żeby model był w stanie modelować dane wejściowe jako sekwencje, w przeciwnym wypadku, pozycja w której umieścimy token/wartość w chwili czasu nie ma żadnego znaczenia.

Nakładamy maskę, żeby wyliczać atencję tylko na podstawie tokenów/chwil czasu, które chcemy, żeby model brał pod uwagę. Np. podczas uczenia generatora tekstu, będziemy maskowali wszystkie następne tokeny, ponieważ nie chcemy, żeby model genrował token na podstawie przyszłych tokenów, natomiast w przypadku klasyfikacji tekstu już taka maska nie jest potrzebna. Podczas trenowania nakłada też się zawsze maskę na tokeny odpowiadające paddingowi.
  

### Bibliografia
#### Sieci rekurencyjne
* [RNN i LSTM](https://colah.github.io/posts/2015-08-Understanding-LSTMs/)
* [ELMo-Embeddingi z kontekstem](https://arxiv.org/abs/1802.05365v2)
* [BERT ELMo zilustrowane](https://jalammar.github.io/illustrated-bert/)
* [Bahdanau Attention](https://arxiv.org/abs/1508.04025)
#### Transformery
* [Artykuł wprowadzający transformery](https://arxiv.org/abs/1706.03762)
* [Wizualizacje działania transformerów](https://jalammar.github.io/illustrated-transformer/)
* [BERT](https://arxiv.org/abs/1810.04805)

## Tematy
### Sub-word tokenizers:
* [Byte-Pair Encoding](https://arxiv.org/abs/1508.07909)
* [WordPiece](https://ai.googleblog.com/2021/12/a-fast-wordpiece-tokenization-system.html)
* [Unigram Language Model](https://arxiv.org/pdf/1804.10959.pdf)
* [SentencePiece](https://jacky2wong.medium.com/understanding-sentencepiece-under-standing-sentence-piece-ac8da59f6b08)
### Embeddingi
* [Word2Vec](https://arxiv.org/abs/1301.3781)
* [GloVe](https://nlp.stanford.edu/projects/glove/)
* [FastText](https://arxiv.org/pdf/1607.04606.pdf)
* [Doc2Vec](https://arxiv.org/pdf/1301.3781.pdf)
## Biblioteki:
* [NLTK](https://www.nltk.org/)
* [Spacy](https://spacy.io/)
* [FastText](https://fasttext.cc/)
* [Hugging face tokenizers](https://huggingface.co/docs/tokenizers/python/latest/)
* [Gensim](https://radimrehurek.com/gensim/)
